In [18]:
import json
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from typing import Dict, List

import nltk
nltk.download('punkt')

def calculate_precision_recall_f1(gold, pred):
    print("gold",gold)
    print("pred",pred)
    if len(pred) == 0:
        return 0, 0, 0
    p = len(gold.intersection(pred)) / len(pred)
    r = len(gold.intersection(pred)) / len(gold)
    f1 = 2 * ((p * r) / (p + r)) if (p + r) > 0 else 0
    return p, r, f1


def get_subject_object_hallucinations(ps, ontology, test_sentence, triples):
    if len(triples) == 0:
        return 0, 0

    # append the test sentence with concepts from the ontology
    #print("test_sentence",test_sentence)
    test_sentence += " ".join([c["label"] for c in ontology['concepts']])
    #print("test_sentence after ",test_sentence)
    # stem each word in the test sentence concatenated with the ontology concepts
    stemmed_sentence = "".join([ps.stem(word) for word in word_tokenize(test_sentence)])
    #print("stemmed_sentence",stemmed_sentence)
    normalized_stemmed_sentence = re.sub(r"(_|\s+)", '', stemmed_sentence).lower()
    #print("normalized_stemmed_sentence",normalized_stemmed_sentence)

    num_subj_hallucinations, num_obj_hallucinations = 0, 0
    for triple in triples:
        normalized_stemmed_subject = clean_entity_string(ps, triple[0])
        normalized_stemmed_object = clean_entity_string(ps, triple[2])

        if normalized_stemmed_sentence.find(normalized_stemmed_subject) == -1:
            num_subj_hallucinations += 1
        if normalized_stemmed_sentence.find(normalized_stemmed_object) == -1:
            num_obj_hallucinations += 1

    subj_hallucination = num_subj_hallucinations / len(triples)
    obj_hallucination = num_obj_hallucinations / len(triples)
    return subj_hallucination, obj_hallucination
    
def get_ontology_conformance(ontology, triples):
    if len(triples) == 0:
        return 1, 0
    ont_rels = [rel['label'].replace(" ", "_") for rel in ontology['relations']]
    print("ont_rels   :",ont_rels)
    num_rels_conformant = len([tr for tr in triples if tr[1] in ont_rels])
    ont_conformance = num_rels_conformant / len(triples)
    rel_hallucination = 1 - ont_conformance
    return ont_conformance, rel_hallucination


def normalize_triple(sub_label, rel_label, obj_label):
    sub_label = re.sub(r"(_|\s+)", '', sub_label).lower()
    rel_label = re.sub(r"(_|\s+)", '', rel_label).lower()
    obj_label = re.sub(r"(_|\s+)", '', obj_label).lower()
    return f"{sub_label}{rel_label}{obj_label}"

def clean_entity_string(ps, entity):
    stemmed_entity = "".join([ps.stem(word) for word in word_tokenize(entity)])
    normalized_stemmed_entity = re.sub(r"(_|\s+)", '', stemmed_entity).lower()
    return normalized_stemmed_entity

def evaluate_and_save_results(ground_truth_data, ontology, model_data, output_file):
    ps = PorterStemmer()
    results = []

    for gt_entry, model_entry in zip(ground_truth_data, model_data):
        # ✅ Skip if ground truth has no triples
        if not gt_entry.get('triples'):
            continue

        gt_triples = [[tr['sub'], tr['rel'], tr['obj']] for tr in gt_entry['triples']]
        system_triples = [[tr['sub'], tr['rel'], tr['obj']] for tr in model_entry['triples']]
        #system_triples = [[tr[0], tr[1], tr[2]] for tr in model_entry['triples']]

        gt_relations = {tr[1].replace(" ", "_") for tr in gt_triples}
        filtered_system_triples = [tr for tr in system_triples if tr[1] in gt_relations]

        normalized_gt_triples = {normalize_triple(tr[0], tr[1], tr[2]) for tr in gt_triples}
        normalized_system_triples = {normalize_triple(tr[0], tr[1], tr[2]) for tr in filtered_system_triples}

        precision, recall, f1 = calculate_precision_recall_f1(normalized_gt_triples, normalized_system_triples)
        ont_conformance, rel_hallucination = get_ontology_conformance(ontology, system_triples)
        subj_hallucination, obj_hallucination = get_subject_object_hallucinations(ps, ontology, gt_entry['sent'], system_triples)

        result = {
            "id": gt_entry['id'],
            "precision": f"{precision:.2f}",
            "recall": f"{recall:.2f}",
            "f1": f"{f1:.2f}",
            "onto_conf": f"{ont_conformance:.2f}",
            "rel_halluc": f"{rel_hallucination:.2f}",
            "sub_halluc": f"{subj_hallucination:.2f}",
            "obj_halluc": f"{obj_hallucination:.2f}",
            "llm_triples": system_triples,
            "filtered_llm_triples": filtered_system_triples,
            "gt_triples": gt_triples,
            "sent": gt_entry['sent']
        }

        results.append(result)

    with open(output_file, "w") as f:
        for result in results:
            f.write(json.dumps(result) + "\n")

def read_ground_truth_jsonl(file_path):
    with open(file_path, "r") as file:
        data = [json.loads(line) for line in file]
        for entry in data:
            if not all(key in entry for key in ['id', 'sent', 'triples']):
                raise ValueError(f"Entry missing required keys: {entry}")
        return data

def read_ontology_json(json_path: str) -> Dict:
    """
    Utility method for reading JSON files
    :param json_path: path to the json file
    :return: json file content as a dictionary
    """
    with open(json_path) as in_file:
        return json.load(in_file)

def read_model_jsonl(file_path):
    with open(file_path, "r") as file:
        data = [json.loads(line) for line in file]
        for entry in data:
            if not all(key in entry for key in ['id', 'triples']):
                raise ValueError(f"Entry missing required keys: {entry}")
        return data

[nltk_data] Downloading package punkt to
[nltk_data]     /upb/users/b/balram/profiles/unix/cs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
output_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/baseline_evaluation_statistics/Llama/without_missing_GT/ont_9_nature_llm_stats_improved.jsonl"
ground_truth_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/ground_truth/ont_9_nature_ground_truth.jsonl"
ontology_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/ontology/9_nature_ontology.json"
model_response_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/response_run2/Llama3/cot_response/ont_9_nature_llm_response_improved.jsonl"



ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)
ontology_data = read_ontology_json(ontology_filepath)  # No slicing needed
model_data = read_model_jsonl(model_response_filepath)

evaluate_and_save_results(ground_truth_data, ontology_data, model_data, output_filepath)


gold {'sacredkingfisherparenttaxontodiramphus'}
pred {'sacredkingfisherparenttaxontodiramphus'}
ont_rels   : ['parent_taxon', 'mountain_range', 'parent_peak', 'taxon_common_name', 'tributary', 'origin_of_the_watercourse', 'mouth_of_the_watercourse', 'IUCN_conservation_status', 'taxon_synonym', 'reservoir_created', 'drainage_basin', 'mountains_classification_', 'Habitat']
gold {'conferencepearparenttaxonpyruscommunis'}
pred set()
ont_rels   : ['parent_taxon', 'mountain_range', 'parent_peak', 'taxon_common_name', 'tributary', 'origin_of_the_watercourse', 'mouth_of_the_watercourse', 'IUCN_conservation_status', 'taxon_synonym', 'reservoir_created', 'drainage_basin', 'mountains_classification_', 'Habitat']
gold {'morphiniparenttaxonmorphinae'}
pred {'morphiniparenttaxonmorphinae'}
ont_rels   : ['parent_taxon', 'mountain_range', 'parent_peak', 'taxon_common_name', 'tributary', 'origin_of_the_watercourse', 'mouth_of_the_watercourse', 'IUCN_conservation_status', 'taxon_synonym', 'reservoir_cre

In [4]:
# Example usage:



output_filepath = "../data/dbpedia_webnig/evaluation_statistics_exp/Llama/without_missing_GT/ont_1_university_llm_stats_improvedbaseline.jsonl"
ground_truth_filepath ="../data/dbpedia_webnig/ground_truth/ont_1_university_ground_truth.jsonl"
ontology_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/dbpedia_webnig/ontology/1_university_ontology.json"
model_response_filepath ="../data/dbpedia_webnig/response/Llama3/cot_response_without_quant_batch/ont_1_university_llm_response_improved.jsonl"


ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)
ontology_data = read_ontology_json(ontology_filepath)  # No slicing needed
model_data = read_model_jsonl(model_response_filepath)

evaluate_and_save_results(ground_truth_data, ontology_data, model_data, output_filepath)

gold {'awhengineeringcollegeacademicstaffsize250', 'awhengineeringcollegestatekerala', 'awhengineeringcollegecity"kuttikkattoor"'}
pred set()
ont_rels   : ['director', 'sportsOffered', 'latinName', 'hasToItsWest', 'numberOfPostgraduateStudents', 'established', 'capital', 'headquarter', 'hasToItsNortheast', 'religion', 'postalCode', 'motto', 'wasGivenTheTechnicalCampusStatusBy', 'location', 'hasToItsNorthwest', 'patronSaint', 'numberOfStudents', 'affiliation', 'anthem', 'nickname', 'river', 'numberOfUndergraduateStudents', 'leaderTitle', 'staff', 'ethnicGroup', 'legislature', 'city', 'leader', 'isPartOf', 'longName', 'academicStaffSize', 'president', 'rector', 'outlookRanking', 'neighboringMunicipality', 'elevationAboveTheSeaLevel', 'campus', 'dean', 'country', 'largestCity', 'governmentType', 'state', 'sportGoverningBody', 'officialSchoolColour', 'founder', 'numberOfDoctoralStudents']
gold {'awhengineeringcollegeacademicstaffsize250', 'awhengineeringcollegeestablished2001', 'awhenginee

In [5]:
START_INDEX = 31
END_INDEX = 33

ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)[START_INDEX:END_INDEX]
ontology_data = read_ontology_json(ontology_filepath)  # No slicing needed
model_data = read_model_jsonl(model_response_filepath)[START_INDEX:END_INDEX]

evaluate_and_save_results(ground_truth_data, ontology_data, model_data, output_filepath)


gold {'acharyainstituteoftechnologyaffiliationvisvesvarayatechnologicaluniversity', 'acharyainstituteoftechnologyestablished2000', 'acharyainstituteoftechnologywasgiventhetechnicalcampusstatusbyallindiacouncilfortechnicaleducation', 'allindiacouncilfortechnicaleducationlocationmumbai', 'acharyainstituteoftechnologycampus"insoldevanahalli,acharyadr.sarvapalliradhakrishnanroad,hessarghattamainroad,bangalore–560090."'}
pred {'acharyainstituteoftechnologyaffiliationvisvesvarayatechnologicaluniversity', 'acharyainstituteoftechnologyestablished2000', 'acharyainstituteoftechnologylocationsoldevanahalli', 'acharyainstituteoftechnologywasgiventhetechnicalcampusstatusbyallindiacouncilfortechnicaleducation', 'acharyainstituteoftechnologycampusacharyadr.sarvapalliradhakrishnanroad'}
ont_rels   : ['director', 'sportsOffered', 'latinName', 'hasToItsWest', 'numberOfPostgraduateStudents', 'established', 'capital', 'headquarter', 'hasToItsNortheast', 'religion', 'postalCode', 'motto', 'wasGivenTheTechn

In [9]:
NUM_EXAMPLES = 3  # Change this to any number of examples you want to run

ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)[:NUM_EXAMPLES]
ontology_data = read_ontology_json(ontology_filepath)  # No slicing here
model_data = read_model_jsonl(model_response_filepath)[:NUM_EXAMPLES]

evaluate_and_save_results(ground_truth_data, ontology_data, model_data, output_filepath)


In [6]:
# Read and evaluate



ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)
ontology_data= read_ontology_json(ontology_filepath)
model_data = read_model_jsonl(model_response_filepath)
evaluate_and_save_results(ground_truth_data,ontology_data, model_data, output_filepath)

In [3]:
# Read and evaluate


NUM_EXAMPLES = 3  # Change to 5 or any number you want

ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)[:NUM_EXAMPLES]
ontology_data= read_ontology_json(ontology_filepath)[:NUM_EXAMPLES]
model_data = read_model_jsonl(model_response_filepath)[:NUM_EXAMPLES]
evaluate_and_save_results(ground_truth_data,ontology_data, model_data, output_filepath)

TypeError: unhashable type: 'slice'

In [ ]:
# Limit number of examples to evaluate
NUM_EXAMPLES = 3  # Change to 5 or any number you want

ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)[:NUM_EXAMPLES]
model_data = read_model_jsonl(model_response_filepath)[:NUM_EXAMPLES]

evaluate_and_save_results(ground_truth_data, model_data, output_filepath)


In [ ]:
Mistral

In [56]:
# 1_movie
output_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/baseline_evaluation_statistics/Mistral/without_missing_GT/ont_1_movie_llm_stats_improved.jsonl"
ground_truth_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/ground_truth/ont_1_movie_ground_truth.jsonl"
ontology_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/ontology/1_movie_ontology.json"
model_response_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/response_run2/Mistral/cot_response/ont_1_movie_llm_response_improved.jsonl"


In [57]:
# Read and evaluate
ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)
ontology_data= read_ontology_json(ontology_filepath)
model_data = read_model_jsonl(model_response_filepath)
evaluate_and_save_results(ground_truth_data,ontology_data, model_data, output_filepath)

In [36]:
# alpha and Vicuna

In [50]:
# 9_nature
output_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/alpha_vicuna/vicuna/evaluation_statics/ont_1_movie_llm_stats.jsonl"
ground_truth_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/ground_truth/ont_1_movie_ground_truth.jsonl"
ontology_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/ontology/1_movie_ontology.json"
model_response_filepath = "/upb/users/b/balram/profiles/unix/cs/Text2KG/withont/data/wikidata/alpha_vicuna/vicuna/response/ont_1_movie_llm_responses.jsonl"


In [51]:
# Read and evaluate
ground_truth_data = read_ground_truth_jsonl(ground_truth_filepath)
ontology_data= read_ontology_json(ontology_filepath)
model_data = read_model_jsonl(model_response_filepath)
evaluate_and_save_results(ground_truth_data,ontology_data, model_data, output_filepath)